Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [91]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np
from IPython.display import Image # import d'images intégrées dans Jupyter.

In [92]:
def readFile(file_name):
    """
    Cette fonction lit un fichier formatté au type utilise dans les exemples du
    projet
    @pre                : Le fichier file_name est organise de la meme maniere
                          que les exemples donnes sur Moodle.
    
    @param file_name    : contient un String avec le nom du fichier a lire.
    
    @return n_points    : Contient un 'int' avec le nombre de points
    @return n_conduites : Contient un 'int' avec le nombre de conduites
    @return X           : Contient un 'numpy.array' avec les abscisses des points.
    @return Y           : Contient un 'numpy.array' avec les ordonnees des points.
    @return Z           : Contient un 'numpy.array' avec la hauteur des points.
    @return A           : Contient un 'numpy.array' avec la matrice d'incidence
                          des conduites.
    @return alpha       : Contient un 'float' avecla valeur de alpha.
    @return R           : Contient un 'float' avec le rayon des conduites.
    @return pa          : Contient un 'numpy.array' avec les points
                          d'approvisionnement.
    @return da_max      : Contient un 'numpy.array' avec les debits maximaux
                          extractibles.
    @return C           : Contient un 'numpy.array' avec les couts d'extraction.
    @return pc          : Contient un 'numpy.array' avec les points de
                          consommation.
    @return dc_min      : Contient un 'numpy.array' avec les debits minimaux
                          consommables.
    @return dc_max      : Contient un 'numpy.array' avec les debits maximaux
                          consommables.
    @return Pf          : Contient un 'float' avec le prix facture.
    @return pi          : Contient un 'numpy.array' avec les points
                          intermediaires.
    @return unit        : Contient un 'String' avec l'unite des coordonnes des
                          points.
    """
    with open(file_name,'r') as f:
        n_points    = (int)(f.readline().split()[0])
        n_conduites = (int)(f.readline().split()[0])
        f.readline()
        unit        = (f.readline().split(" = ")[1])
        
        coord       = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        X           = coord[:,0]
        Y           = coord[:,1]
        Z           = coord[:,2]
        
        f.readline()
        f.readline()
        A           = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        f.readline()
        
        UL_alpha    = f.readline().split(" = ")[1]
        UL_alpha    = UL_alpha.split()[0]
        UL2_alpha   = np.array(UL_alpha.split("^"))
        if(len(UL2_alpha) == 1):
            alpha   = (float)(UL2_alpha)
        else:
            alpha   = (float)(UL2_alpha[0]) ** (float)(UL2_alpha[1])
        
        R           = (float)(f.readline().split()[4])
        f.readline()
        
        UL_pa       = f.readline().split(" = [")[1]
        UL_pa       = UL_pa.split("]")[0]
        pa          = np.array(list(int(w) for w in UL_pa.split()[:]))
        
        UL_da_max   = f.readline().split(" = [")[1]
        UL_da_max   = UL_da_max.split("]")[0]
        da_max      = np.array(list(float(w) for w in UL_da_max.split()[:]))
        
        UL_C        = f.readline().split(" = [")[1]
        UL_C        = UL_C.split("]")[0]
        C           = np.array(list(float(w) for w in UL_C.split()[:]))
        
        f.readline()
        
        UL_pc       = f.readline().split(" = [")[1]
        UL_pc       = UL_pc.split("]")[0]
        pc          = np.array(list(int(w) for w in UL_pc.split()[:]))
        
        UL_dc_min   = f.readline().split(" = [")[1]
        UL_dc_min   = UL_dc_min.split("]")[0]
        dc_min      = np.array(list(float(w) for w in UL_dc_min.split()[:]))
        
        UL_dc_max   = f.readline().split(" = [")[1]
        UL_dc_max   = UL_dc_max.split("]")[0]
        dc_max      = np.array(list(float(w) for w in UL_dc_max.split()[:]))
        
        Pf          = (float)(f.readline().split()[3])
        
        f.readline()
        
        UL_pi       = f.readline().split("[")[1]
        UL_pi       = UL_pi.split("]")[0]
        pi          = np.array(list(int(w) for w in UL_pi.split()[:]))
        
    return (n_points, n_conduites, X, Y, Z, A, alpha, R, pa, da_max, C, pc, dc_min, dc_max, Pf, pi, unit)

In [93]:
def readSecondFile(filename):
    """
    Cette fonction permet de récupérer les données stockées dans le fichier 'partie2'
    @pre : filename est un String pointant vers un fichier.
    @pre : le fichier 'filename' est formaté correctement.
    @pre : le cout de construction d'un chateau d'eau est exprime en millions.
    
    @return Cchateau contient le cout de construction d'un chateau d'eau (€/m).
    @return Hmax     contient la hauteur max. d'un chateau d'eau (metre).
    @return Btot     contient le budget total de construction des chateaux.
    @return Ccc      contient le cout de construction d'une conduite.
    """
    with open(filename,'r') as f:
        f.readline()
        f.readline()
        Cchateau = (float)(f.readline().split()[7])*1000000
        
        UL_Hmax  = (f.readline().split()[6]).split(",")
        if(len(UL_Hmax)==2):
            Hmax = (float)(UL_Hmax[0]) + (float)(UL_Hmax[1])/(10**(len(UL_Hmax[1])))
        else:
            Hmax = (float)(UL_Hmax[0])
        Btot     = (float)(f.readline().split()[7])
        f.readline()
        Ccc      = (float)(f.readline().split()[6])
    return (Cchateau, Hmax, Btot, Ccc)

# Programme 1

In [94]:
def Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha):
    """
    Cette fonction maximise le benefice realise par le reseau fourni.
    
    @param nPoints        : Contient un 'int' avec le nombre de points.
    @param nConduites     : Contient un 'int' avec le nombre de conduites.
    @param X              : Contient un 'numpy.array' avec les abscisses des points.
    @param Y              : Contient un 'numpy.array' avec les ordonnees des points.
    @param Z              : Contient un 'numpy.array' avec la hauteur des points.
    @param A              : Contient un 'numpy.array' avec la matrice d'incidence des conduites.
    @param R              : Contient un 'float' avec le rayon des conduites.
    @param Pa             : Contient un 'numpy.array' avec les points d'approvisionnement.
    @param Damax          : Contient un 'numpy.array' avec les debits maximaux extractibles.
    @param C              : Contient un 'numpy.array' avec les couts d'extraction.
    @param Pc             : Contient un 'numpy.array' avec les points de consommation.
    @param Dcmin          : Contient un 'numpy.array' avec les debits minimaux consommables.
    @param Dcmax          : Contient un 'numpy.array' avec les debits maximaux consommables.
    @param Pf             : Contient un 'float' avec le prix facture.
    @param Pi             : Contient un 'numpy.array' avec les points intermediaires.
    @param alpha          : Contient un 'float' avec la valeur de alpha.
    
    @return (-1)*res.fun  : Contient un 'float' avec la valeur maximale du benefice realise par le reseau fourni.
    @return DebC          : Contient un 'numpy.array' avec les debits optimaux extraits aux points de consommation.
    @return DebA          : Contient un 'numpy.array' avec les debits optimaux fournis par les points d'approvisionnement.
    """
    
    h = abs(A.T@Z)            # Denivele de chaque conduite entre le point de depart et d'arrivee.
    x = abs(A.T@X)            # Difference de x de chaque conduite entre le point de depart et d'arrivee.
    y = abs(A.T@Y)            # Difference de y de chaque conduite entre le point de depart et d'arrivee.
    L = np.sqrt(h*h+x*x+y*y)  # Longueur de chaque conduite  
    
    A1     = np.concatenate((A[np.array(Pc)-1],-1* A[np.array(Pa)-1]))   # Tous les debits doivent etre positifs.
    A3     = np.concatenate((A[np.array(Pc)-1], A[np.array(Pa)-1]))      # On laisse les debits aux points d'approvisionnement 
                                                                         # negatifs.
    DebMax = alpha*R*R*h/L*A1    # Debit max, juste sur les points pas intermediaires.
    
    A2     = A[Pc-1]             # Uniquement les points de consommation.
    DebMin = -1*alpha*R*R*h/L*A2 # Debit min
    
    Aub    = np.concatenate((DebMax,DebMin))
    Argent = np.concatenate((Pf,C))
    Eye    = np.eye(len(Argent))*Argent
    c      = -1*np.matmul(Argent,A3)*(alpha*R*R*h/L)    
   
    A5     = A[np.array(Pi)-1]  # On ne prend que les points intermediaires.
    Aeq    = A5*alpha*R*R*h/L   # Points intemediaires ont un débit resultant nul.
    beq    = np.zeros(len(Pi))
    b      = np.concatenate((np.concatenate((Dcmax, Damax)), -1*Dcmin))
    
    bounds = np.concatenate((np.zeros((nConduites,1)),np.ones((nConduites,1))),axis=1) # Limites (vecteur de (0,1) sur le nombre de conduites)
    res    = linprog(c, A_ub = Aub, b_ub = b,A_eq=Aeq, b_eq=beq,bounds=bounds,options={'tol': 1e-8})
    DebC   = (A2*alpha*R*R*h/L)@res.x
    A4     = A[Pa-1]
    DebA   = (A4*alpha*R*R*h/L)@res.x
    Prix   = Pf.T@DebC
    Couts  = C.T@DebA
    
    return (-1*res.fun,DebC,DebA)

# Programme 2

In [95]:
def Ameliorations(nPoints,nConduites,X,Y,Z,A,alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi): 
    """
    Cette fonction maximise le benefice realise par le reseau fourni lorsque les points de consommation peuvent depasser
    leur demande maximale initiale, avec diminution du prix.
    
    @param nPoints        : Contient un 'int' avec le nombre de points.
    @param nConduites     : Contient un 'int' avec le nombre de conduites.
    @param X              : Contient un 'numpy.array' avec les abscisses des points.
    @param Y              : Contient un 'numpy.array' avec les ordonnees des points.
    @param Z              : Contient un 'numpy.array' avec la hauteur des points.
    @param A              : Contient un 'numpy.array' avec la matrice d'incidence des conduites.
    @param alpha          : Contient un 'float' avec la valeur de alpha.
    @param R              : Contient un 'float' avec le rayon des conduites.
    @param Pa             : Contient un 'numpy.array' avec les points d'approvisionnement.
    @param Damax          : Contient un 'numpy.array' avec les debits maximaux extractibles.
    @param C              : Contient un 'numpy.array' avec les couts d'extraction.
    @param Pc             : Contient un 'numpy.array' avec les points de consommation.
    @param Dcmin          : Contient un 'numpy.array' avec les debits minimaux consommables.
    @param Dcmax          : Contient un 'numpy.array' avec les debits maximaux consommables.
    @param Pf             : Contient un 'float' avec le prix facture.
    @param Pi             : Contient un 'numpy.array' avec les points intermediaires.
    
    @return (-1)*res.fun  : Contient un 'float' avec la valeur maximale du benefice realise par le reseau fourni.
    @return DebitC        : Contient un 'numpy.array' avec les debits optimaux extraits aux points de consommation.
    @return DebitA        : Contient un 'numpy.array' avec les debits optimaux fournis par les points d'approvisionnement.
    """
    
    h = abs(A.T@Z)           # Denivele de chaque conduite entre le point de depart et d'arrivee.
    x = abs(A.T@X)           # Difference de x de chaque conduite entre le point de depart et d'arrivee.
    y = abs(A.T@Y)           # Difference de y de chaque conduite entre le point de depart et d'arrivee.
    L = np.sqrt(h*h+x*x+y*y)
    
    Ac = A[Pc-1] # On prend les lignes de la matrice d'incidence relatives aux points de consommation
    Aa = A[Pa-1] # On prend les lignes de la matrice d'incidence relatives aux points d'approvisionnement
    Ai = A[Pi-1] # On prend les lignes de la matrice d'incidence relatives aux points intermediaires
    
    debit = alpha*R*R*h/L # Formule de debit max (theta = 1)
    
    Argent= np.concatenate([Pf,C,Pf/2])
    
    grandA= np.concatenate([np.concatenate([Ac,Aa,np.zeros((len(Pc),nConduites))],axis=0)*debit,np.concatenate([np.zeros((len(Pc),nConduites)),Aa,Ac],axis=0)*debit],axis=1)
    
    c     = -1*np.matmul(Argent,grandA)                              # Fonction objectif
    
    A1    = np.concatenate([Ac*debit,np.zeros(np.shape(Ac))],axis=1) # Dc < Dcmax et -Dc < -Dcmin
    
    Atrop = np.concatenate([np.zeros(np.shape(Ac)),Ac*debit],axis=1) # Dctrop < 0,25*Dcmax
    
    Aapp  = np.concatenate([-1*Aa*debit,-1*Aa*debit],axis=1)         # Da < Damax
    
    theta = np.concatenate([np.eye(nConduites),np.eye(nConduites)],axis=1)        # theta1 + theta2 < 1
    Aub   = np.concatenate([A1,-1*A1,Atrop,Aapp,theta])                           # Concatenate de la grande matrice Aub
    b     = np.concatenate([Dcmax,-1*Dcmin,0.25*Dcmax,Damax,np.ones(nConduites)]) # Concatenate du grand vecteur de contraintes
    
    Aeq   = np.concatenate([Ai*debit,Ai*debit],axis=1) # Di = 0
    beq   = np.zeros(len(Pi))                          # Di = 0
    Bounds= (0,1)                                      # theta > 0
    
    res   = linprog(c, A_ub = Aub, b_ub = b,A_eq=Aeq, b_eq=beq,bounds=Bounds,options={'tol': 1e-12})#Resolution

    theta = res.x[0:nConduites]+res.x[nConduites:2*nConduites]

    DebitCok  = (Ac*debit)@res.x[0:nConduites]
    DebitCtrop= (Ac*debit)@res.x[nConduites:2*nConduites]
    DebitC    = (Ac*debit)@theta
    DebitA    = (-1*Aa*debit)@theta
    DebitI    = (Ai*debit)@theta
    fonction  = Pf.T@DebitCok+(Pf/2).T@DebitCtrop-C@DebitA
    
    return (-1*res.fun,DebitC,DebitA)

# Programme 3

In [96]:
def AmeliorationsTour(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,Cchateau, hmax, Btot): 
    hmax=hmax*1e-3
    Cchateau=Cchateau*1e3
    Pannee=365*10*24
    """ Calcul de la longueur """
    h = abs(A.T@Z)        # le denivele de chaque conduite entre le pt de depart et d'arrivee.
    x = abs(A.T@X)        # la difference de x de chaque conduite entre le pt de depart et d'arrivee.
    y = abs(A.T@Y)        # la difference de y de chaque conduite entre le pt de depart et d'arrivee.
    L=np.sqrt(h*h+x*x+y*y)# On suppose que L ne change pas si on construit un chateau d'eau
    """ Repartition des points """
    Ac=A[Pc-1]            # On prend les lignes de la matrice d'incidence relatives aux points de consommation
    Aa=A[Pa-1]            # On prend les lignes de la matrice d'incidence relatives aux points d'approvisionnement
    Ai=A[Pi-1]            # On prend les lignes de la matrice d'incidence relatives aux points intermediaires
    """ Calcul du debit """
    debit  = np.outer(alpha*R*R/L,np.ones(len(Pa))) # Formule de debit sans hauteur et sans theta
    Argent = np.concatenate([Pf*Pannee,Pf/2*Pannee,C*Pannee,Cchateau*np.ones(len(Pa))])
    AFlux1 = np.concatenate([Ac,np.zeros((len(Pc),nConduites)),Aa,np.zeros((len(Pa),nConduites))])
    AFlux2 = np.concatenate([np.zeros((len(Pc),nConduites)),Ac,Aa,np.zeros((len(Pa),nConduites))])
    AHsup  = np.concatenate([np.zeros((len(Pc)*2+len(Pa),len(Pa))),-1*np.eye(len(Pa))])
    grandA = np.concatenate([AFlux1,AFlux2,AHsup],axis=1)
    c      = -1*np.matmul(Argent,grandA)#fonction objectif
    
    A1     = np.concatenate([Ac,np.zeros((len(Ac),nConduites+len(Pa)))],axis=1)#Dc<Dcmax et -Dc<-Dcmin
    Atrop  = np.concatenate([np.zeros((len(Ac),len(Ac[0]))),Ac,np.zeros((len(Ac),len(Pa)))],axis=1)#Dctrop<0,25*Dcmax
    Aapp   = np.concatenate([-1*Aa,-1*Aa,np.zeros((len(Aa),len(Pa)))],axis=1)#Da<Damax    
    flux   = np.concatenate([np.eye(nConduites),np.eye(nConduites),debit*Aa.T],axis=1)#flux1+flux2-alpha*R*R*hsup/L<alpha*R*R*h/L
    htour  = np.concatenate([np.zeros((len(Pa),2*nConduites)),np.eye(len(Pa))],axis=1)
    hbudget1 = np.zeros((1,2*nConduites))
    hbudget2 = Cchateau*np.ones((1,len(Pa)))
    hbudget = np.concatenate([hbudget1,hbudget2],axis=1)
    
    Aub    = np.concatenate([A1,-1*A1,Atrop,Aapp,flux,htour,hbudget,-1*Atrop,-1*Aapp]) #Concatenate de la grande matrice Aub
    b      = np.concatenate([Dcmax,-1*Dcmin,0.25*Dcmax,Damax,alpha*R*R*(h)/L,np.ones(len(Pa))*hmax,[Btot],np.zeros(len(Pc)),np.zeros(len(Pa))])#Concatenate du grand vecteur de contraintes
    
    Aeq    = np.concatenate([Ai,Ai,np.zeros((len(Pi),len(Pa)))],axis=1)#Di=0
    beq    = np.zeros(len(Pi))#Di=0
    Bounds = (0,None)#theta>0
    res    = linprog(c, A_ub = Aub, b_ub = b,A_eq=Aeq, b_eq=beq,bounds=Bounds,options={'tol': 1e-12})#Resolution
    #print(res)
    hsupp =res.x[(2*nConduites):(2*nConduites+len(Pa))]
    flux1=res.x[0:nConduites]
    flux2=res.x[nConduites:2*nConduites]
    flux=flux1+flux2
    DebitCok=Ac@flux1
    DebitCtrop=Ac@flux2
    DebitC=Ac@flux
    DebitA=-1*Aa@flux
    DebitI=Ai@flux
    fonction = Pannee*Pf.T@DebitCok+Pannee*(Pf/2).T@DebitCtrop-Pannee*C@DebitA-(Cchateau*np.ones(len(Pa)))@hsupp
    
    return (-1*res.fun,DebitCok,DebitCtrop,DebitC,DebitA,DebitI,hsupp,fonction)

# Lecture de données

In [97]:
#filename1 = "Données/Data-partie1.txt"
filename1 = "OurReseau/Tiny"
#filename2 = "Données/Data-parties2et3.txt"
filename2 = "OurReseau/Tiny2.txt"
(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,unit) = readFile(filename1)
(Cchateau, Hmax, Btot, Ccc) = readSecondFile(filename2)

# Runs

In [98]:
rep1 = Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Pc)),Pi,alpha)
rep2 = Ameliorations(nPoints,nConduites,X,Y,Z,A,alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Pc)),Pi)
rep3 = AmeliorationsTour(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Pc)),Pi,Cchateau, Hmax, Btot)

# Prints

In [99]:
print("Program 1 :")
for i in range(len(rep1)):
    print("    - arg {} :".format(i+1))
    print(rep1[i])
    print()
    
print("Program 2 :")
for i in range(len(rep2)):
    print("    - arg {} :".format(i+1))
    print(rep2[i])
    print()
    
print("Program 3 :")
for i in range(len(rep3)):
    print("    - arg {} :".format(i+1))
    print(rep3[i])
    print()

Program 1 :
    - arg 1 :
1.75

    - arg 2 :
[1.5 1. ]

    - arg 3 :
[-2.5]

Program 2 :
    - arg 1 :
1.8749999999999998

    - arg 2 :
[1.875 1.125]

    - arg 3 :
[3.]

Program 3 :
    - arg 1 :
164250.0

    - arg 2 :
[1.5 1. ]

    - arg 3 :
[0.375 0.125]

    - arg 4 :
[1.875 1.125]

    - arg 5 :
[3.]

    - arg 6 :
[0.]

    - arg 7 :
[0.]

    - arg 8 :
164250.0



# Programme 5

In [112]:
def Final(nPoints, X, Y, Z, A, alpha, Pa, Damax, C, Pc, Dcmin, Dcmax, Pf, Pi, Btot, Ccc):
    Conduites_possibles = (int)((nPoints-1)*(nPoints)/2)
    
    """ Creation de la matrice d'incidence """
    A              = np.zeros((nPoints, Conduites_possibles))    
    bloc           = nPoints-1
    indice_ligne   = 0
    indice_colonne = 0
    while(indice_ligne < nPoints - 1):
        for j in range (bloc):
            A[indice_ligne][indice_colonne + j] = 1                                                        # ligne de 1
        A[indice_ligne + 1: indice_ligne + bloc + 1, indice_colonne: indice_colonne + bloc] = np.eye(bloc) # mat identité
        indice_ligne   += 1
        indice_colonne += bloc
        bloc           -= 1
        
    """ Placement des points de production/intermediaires/consommation"""
    for i in range(Conduites_possibles):
        if(Z[(int)(Indices[0][0])] > Z[(int)(Indices[0][1])]): # Si le premier point est le plus haut.
            A[(int)(Indices[0][0])][i] = -1
            
        elif(Z[(int)(Indices[0][0])] < Z[Indices[0][1]]):      # Si le second point est le plus haut.
            A[Indices[0][1]][i] = -1
            
        else:                                                  # Si les deux points sont à la même hauteur
            A[Indices[0][0]][i] = 0
            A[Indices[0][1]][i] = 0
    return A

In [114]:
print(Final(nPoints, X, Y, Z, A, alpha, Pa, Damax, C, Pc, Dcmin, Dcmax, Pf, Pi, Btot, Ccc))

This is A
[[1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 1. 0. 1.]
 [0. 0. 1. 0. 1. 1.]]

This is Z
[2. 1. 0. 0.]

This is A[:,i]
[1. 1. 0. 0.]

This is Indices
[0 1]

This is A with change
[[-1.  1.  1.  0.  0.  0.]
 [ 1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  1.  0.  1.]
 [ 0.  0.  1.  0.  1.  1.]]
This is A[:,i]
[1. 0. 1. 0.]

This is Indices
[0 2]

This is A with change
[[-1. -1.  1.  0.  0.  0.]
 [ 1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  1.  0.  1.]
 [ 0.  0.  1.  0.  1.  1.]]
This is A[:,i]
[1. 0. 0. 1.]

This is Indices
[0 3]

This is A with change
[[-1. -1. -1.  0.  0.  0.]
 [ 1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  1.  0.  1.]
 [ 0.  0.  1.  0.  1.  1.]]
This is A[:,i]
[0. 1. 1. 0.]

This is Indices
[1 2]

This is A with change
[[-1. -1. -1.  0.  0.  0.]
 [ 1.  0.  0. -1.  1.  0.]
 [ 0.  1.  0.  1.  0.  1.]
 [ 0.  0.  1.  0.  1.  1.]]
This is A[:,i]
[0. 1. 0. 1.]

This is Indices
[1 3]

This is A with change
[[-1. -1. -1.  0.  0.  0.]
 [ 1.  0.  0. -1. -1.  0.]
 [ 0.  1.  0. 

## Branch and bound
Première image
<img src="src/Beb_1.png" width="600">
Seconde image
<img src="src/Beb_2.png" width="600">
Troisième image
<img src="src/Beb_3.png" width="600">
Quatrième image
<img src="src/Beb_4.png" width="600">
Cinquième image
<img src="src/Beb_5.png" width="600">

In [ ]:
A = [1, 2, 3]
B = [A, [4, 5, 6], [7, 8, 9]]

print(B[1:2][:])